In [1]:
import requests
import os
from google.colab import userdata
from IPython.display import display, HTML

# Step 1: Retrieve USER and PASSWORD secrets from Colab
username = userdata.get("DEMO_USER")
password = userdata.get("DEMO_PASSWORD")

if not username or not password:
    raise ValueError("Secrets USER and PASSWORD must be set in Colab.")

def print_color(message, color):
    display(HTML(f'<p style="color: {color};">{message}</p>'))

# Step 2: Define the authentication function
def authenticate(username, password):
    """
    Authenticates with the Reveal API using username and password.
    Returns loginSessionId and userId if authentication is successful.
    """
    login_url = "https://consulting.us-east-1.reveal11.cloud/rest/api/v2/login"
    login_data = {
        "username": username,
        "password": password
    }

    # Define headers for the POST request
    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Reveal-API-Tester/1.0"
    }

    # Send POST request to login
    response = requests.post(login_url, json=login_data, headers=headers)
    response.raise_for_status()  # Raise an error if the request fails

    # Parse the response to get loginSessionId and userId
    login_response = response.json()
    login_session_id = login_response.get("loginSessionId")
    user_id = login_response.get("userId")

    if not login_session_id or not user_id:
        raise ValueError("Failed to retrieve loginSessionId and userId from login response.")

    print("Authentication successful!")
    return login_session_id, user_id

# Step 3: Define the function to retrieve projects
def get_projects(login_session_id, user_id):
    """
    Retrieves a list of projects for the authenticated user and prints each project name.
    """
    projects_url = "https://consulting.us-east-1.reveal11.cloud/rest/api/v2/projects"

    # Set headers for the GET request, including the incontrolauthtoken
    headers = {
        "Authorization": f"Bearer {login_session_id}",
        "incontrolauthtoken": login_session_id,
        "User-Agent": "Reveal-API-Tester/1.0",
        "Accept": "application/json"
    }

    # Parameters for GET request
    params = {"userId": user_id}

    # Send GET request to retrieve projects
    response = requests.get(projects_url, headers=headers, params=params)
    response.raise_for_status()  # Raise an error if the request fails

    # Parse the response to get project list
    projects_response = response.json()
    projects = projects_response.get("results", [])

    # Print each project name
    if projects:
        print("Project Names:")
        for project in projects:
            print(project.get("projectName", "Unnamed Project"))
    else:
        print("No projects found.")

# Main Execution
try:
    # Step 1: Authenticate to get session ID and user ID
    session_id, user_id = authenticate(username, password)

    # Step 2: Retrieve and print project names
    get_projects(session_id, user_id)

except requests.exceptions.RequestException as e:
    print_color(f"API request failed: {e}", "red")
except ValueError as e:
    print_color(f"Error: {e}", "red")


Authentication successful!
Project Names:
Enron CommViz v11 yc
Enron Summit 2024 
